yes 24 크롤링

In [35]:
#webdriver-manager 라이브러리 불러오기
from webdriver_manager.chrome import ChromeDriverManager
ChromeDriverManager().install()

'/Users/choiseungmin/.wdm/drivers/chromedriver/mac64/134.0.6998.165/chromedriver-mac-arm64/chromedriver'

In [36]:
from selenium import webdriver

browser = webdriver.Chrome()

In [6]:
url = 'https://www.google.com'
browser.get(url)

In [12]:
from selenium.webdriver.common.by import By

browser.find_element(By.CLASS_NAME, 'uU7dJb').text


'대한민국'

In [37]:
import time

link_list = []
for i in range(1,4):
    print("*"*10,f"현재 {i} 페이지 수집 중입니다.","*"*10)
    url = f"https://www.yes24.com/product/category/bestseller?categoryNumber=001&pageNumber={i}&pageSize=24"
    browser.get(url)

    datas = browser.find_elements(By.CLASS_NAME, 'gd_name')

    for i in datas:
        link = i.get_attribute('href')
        link_list.append(link)

    time.sleep(3)


# print(link_list)

********** 현재 1 페이지 수집 중입니다. **********
********** 현재 2 페이지 수집 중입니다. **********
********** 현재 3 페이지 수집 중입니다. **********


In [44]:
# for link in link_list:
#     browser.get(link)

browser.get(link_list[0])

title = browser.find_element(By.CLASS_NAME, 'gd_name').text
author = browser.find_element(By.CLASS_NAME, 'gd_auth').text
publisher = browser.find_element(By.CLASS_NAME, 'gd_pub').text
publishing = browser.find_element(By.CLASS_NAME, 'gd_date').text
rating = browser.find_element(By.CLASS_NAME, 'yes_b').text
review = browser.find_element(By.CLASS_NAME, 'txC_blue').text
sales = browser.find_element(By.CLASS_NAME, 'gd_sellNum').text.split(" ")[2]
price = browser.find_element(By.CLASS_NAME, 'yes_m').text[:-1]
ranking = browser.find_element(By.CLASS_NAME, 'gd_best').text.split(" | ")[0].split(" ")[2][:-1]
ranking_weeks = browser.find_element(By.CLASS_NAME, 'gd_best').text.split(" | ")[1].split(" ")[2][:-1]

ranking_weeks



'1'

In [45]:
import pymysql
import time
import re
from datetime import datetime
conn = pymysql.connect(
    host='localhost',
    user='root',
    password='jssd23!!',
    db='yes24',
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor
)

with conn.cursor() as cur:
    for link in link_list:
        browser.get(link)

        title = browser.find_element(By.CLASS_NAME, 'gd_name').text
        author = browser.find_element(By.CLASS_NAME, 'gd_auth').text
        publisher = browser.find_element(By.CLASS_NAME, 'gd_pub').text

        publishing = browser.find_element(By.CLASS_NAME, 'gd_date').text
        match = re.search(r'(\d+)년 (\d+)월 (\d+)일', publishing)
        if match:
            year, month, day = match.groups()
            data_obj = datetime(int(year), int(month), int(day))
            publishing = data_obj.strftime("%Y=%m-%d")
        else:
            publishing = "2025-01-01"

        rating = browser.find_element(By.CLASS_NAME, 'yes_b').text
        review = browser.find_element(By.CLASS_NAME, 'txC_blue').text
        review = int(review.replace(",",""))

        sales = browser.find_element(By.CLASS_NAME, 'gd_sellNum').text.split(" ")[2]
        sales = int(sales.replace(",",""))

        price = browser.find_element(By.CLASS_NAME, 'yes_m').text[:-1]
        price = int(price.replace(",",""))


        
        try:
            full_text = browser.find_element(By.CLASS_NAME, 'gd_best')
            parts = full_text.split(" | ")
            ranking_part = parts[0]
            ranking = ''.join(filter(str.isdigit, ranking_part))
        except:
            ranking = 0
        try:
            full_text = browser.find_element(By.CLASS_NAME, 'gd_best')
            parts = full_text.split(" | ")
            ranking_weeks_part = parts[1]
            ranking_weeks = ''.join(filter(str.isdigit, ranking_weeks_part.split()[-1]))
        except:
            ranking_weeks = 0
        # ranking = browser.find_element(By.CLASS_NAME, 'gd_best').text.split(" | ")[0].split(" ")[2][:-1]
        # ranking_weeks = browser.find_element(By.CLASS_NAME, 'gd_best').text.split(" | ")[1].split(" ")[2][:-1]

        sql = "INSERT INTO Books(title, author, publisher, publishing, rating, review, sales, price, ranking, ranking_weeks)\
            VALUES(" "%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        
        cur.execute(sql, (title, author, publisher, publishing, rating, review, sales, price, ranking, ranking_weeks))
        conn.commit()

        time.sleep(2)


ValueError: invalid literal for int() with base 10: '함께 산 책'